<a href="https://colab.research.google.com/github/h30e07/E4sosei/blob/main/tyukan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def read_url(url):
  r = requests.get(url)
  print(r.text)
  soup = BeautifulSoup(r.text, "lxml")
  h4_list = soup.find_all('h4')
  for h4 in h4_list:
    return h4.text

def all_subject_kanrensei():
  '''
  全ての2ペアの科目の関連性を数値化
  最大1
  これより科目間を線で結ぶか結ばないかを判定
  線を引くかどうかの基準の数値は自分で設定(いい感じになるように)
  '''

def kanren_word():
  '''
  関連していそうな単語全てを、ファクターに通して、重要度を数値化
  最大1
  ファクターどうしよう。。。
  重要度が自分で設定した数値を超えていたら表示
  '''

def two_shirabasu_kanrendo():
  from gensim.models.word2vec import Word2Vec

  path = "/content/drive/MyDrive/sosei/latest-ja-word2vec-gensim-model/word2vec.gensim.model"
  w2v_model = Word2Vec.load(path)

  sa = "hoge"
  sb = "hoge"
  sa_words = sa.split()
  sb_words = sb.split()
  sum = 0
  for sa_word in sa_words:
    for sb_word in sb_words:
      sum += model.similarity(sa_word, sb_word)
  output = sum / (len(sa_words) + len(sb_words))

  return output


FileNotFoundError: ignored

# **２つのシラバスの関連度を求める**

In [ ]:
from gensim.models.word2vec import Word2Vec
import glob
import pprint
from google.colab import files
import csv
import time
import json
import itertools


def create_csv(list2):
  with open(output_csv_path, 'w', encoding="utf_8_sig") as fw:
    writer = csv.writer(fw)
    writer.writerow(['s1', 's2', 'relevance'])
    for one in list2:
      writer.writerow(one)

def tow_s_similarity(w2v_model, sa_path, sb_path):
  '''
  2つのシラバスの関連度を0から1の範囲で出力
  作成済みword2vecモデルを使用
  '''
  with open(sa_path) as s1:
    sa = s1.read()
  with open(sb_path) as s2:
    sb = s2.read()
  sa_words = sa.split()
  sb_words = sb.split()
  sum = 0

  bunpu = []
  for sa_word in sa_words:
    for sb_word in sb_words:
      try:
        similarity = w2v_model.similarity(sa_word, sb_word)
        bunpu.append([float(similarity), sa_word, sb_word])
        sum += similarity
      except KeyError:
        sum += 0
  output = sum / (len(sa_words) * len(sb_words))
  bunpu1 = sorted(bunpu, key=lambda x: x[0], reverse=True)
  with open(f"/content/drive/MyDrive/sosei/bunpu.json", 'w') as f:
    json.dump(bunpu1, f, indent=2, ensure_ascii=False)
  return output

def get_ignore_kumiawase(json):
  ignore_kumiawase = []
  for tt in json:
    ignore_kumiawase.append((tt[0], tt[1]))
  return ignore_kumiawase

def get_jissai_kumiawase(all_kumiawase, ignore_kumiawase):
  jissai_kumiawase = []
  branch = "ok"
  for ttt in all_kumiawase:
    for ii in ignore_kumiawase:
      branch = "ok"
      if set(ttt) == set(ii):
        branch = "ignore"
        break
    if branch == "ok":
      jissai_kumiawase.append(ttt)
  return jissai_kumiawase


if __name__ == '__main__':
  sosei_root = "/content/drive/MyDrive/sosei"
  w2v_model_path = f"{sosei_root}/Pre-Trained Word2Vec Models/latest-ja-word2vec-gensim-model/word2vec.gensim.model"

  w2v_model = Word2Vec.load(w2v_model_path)

  files = glob.glob(f"{sosei_root}/meishi/*")
  all_kumiawase = list(itertools.combinations(files,2))

  output_csv_path = f"{sosei_root}/seido1.csv"
  '''
  json_open = open(f"{sosei_root}/a.json", 'r')
  output = json.load(json_open)

  ignore_kumiawase = get_ignore_kumiawase(output)
  jissai_kumiawase = get_jissai_kumiawase(all_kumiawase, ignore_kumiawase)

  print(len(jissai_kumiawase))
  '''
  tow_s_similarity(w2v_model, "/content/drive/MyDrive/sosei/meishi/meishi_1基礎数学D(0077).txt", "/content/drive/MyDrive/sosei/meishi/meishi_3ディジタル回路ⅠA(2093).txt")
  '''
  limit_minute = 30 #mm分でプログラムを終わらせて途中経過をファイルに保存
  t1 = time.time()
  for one in jissai_kumiawase:
    relevance = tow_s_similarity(w2v_model, one[0], one[1])
    output.append([one[0], one[1], relevance])

    t2 = time.time()
    elapsed_time = t2- t1
    if elapsed_time > limit_minute * 60:
      break

  with open(f"{sosei_root}/a.json", 'w') as f:
    json.dump(output, f, indent=2, ensure_ascii=False)
  '''


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).


出来上がった全科目の関連性を関連度で並び替え

In [ ]:
import json
json_path = "/content/drive/MyDrive/sosei/a.json"
json_open = open(json_path, 'r')
output = json.load(json_open)
print(len(output))

990


正解データのcsvファイル読み込みand科目の組み合わせのみをまとめたjsonファイルを出力

In [ ]:
import csv
import pprint
import json

l = []
with open('/content/drive/MyDrive/sosei/map.csv') as f:
  reader = csv.reader(f)
  for i, row in enumerate(reader):
    if i == 0:
      continue
    l.append([row[1], row[2]])
pprint.pprint(l)
with open(f"/content/drive/MyDrive/sosei/map_2kamoku.json", 'w') as f:
  json.dump(l, f, indent=2, ensure_ascii=False)

[['プログラミング', 'コンピュータグラフィクス'],
 ['プログラミング', 'プログラミングⅡ'],
 ['電気基礎', '電気基礎Ⅱ'],
 ['基礎数学F', 'メカニズム設計概論'],
 ['基礎数学E', 'ディジタル回路ⅠA'],
 ['力学', '力学Ⅱ'],
 ['線形代数A', 'エネルギー変換工学A'],
 ['線形代数A', 'エネルギー変換工学B'],
 ['線形代数A', '電磁気学Ⅰ'],
 ['線形代数B', 'コンピュータグラフィクス'],
 ['微分積分学ⅠA', '電磁気学Ⅰ'],
 ['微分積分学ⅠB', '電磁気学Ⅰ'],
 ['微分積分学ⅠB', '応用物理ⅠA'],
 ['力学Ⅱ', '電磁気学Ⅰ'],
 ['プログラミングⅡ', 'ロボットエレクトロニクス'],
 ['電気基礎Ⅱ', '電気回路ⅠA'],
 ['電気基礎Ⅱ', '電気回路ⅠB'],
 ['電気基礎Ⅱ', '電磁気学Ⅰ'],
 ['電気基礎Ⅱ', 'エネルギー変換工学A'],
 ['電気基礎Ⅱ', 'エネルギー変換工学B'],
 ['微分積分学ⅡB', '電磁気学Ⅰ'],
 ['応用物理ⅠA', 'メカニズム設計概論'],
 ['電気回路ⅠA', 'エネルギー変換工学A'],
 ['電気回路ⅠB', 'エネルギー変換工学A'],
 ['電気回路ⅠB', 'エネルギー変換工学B']]


In [ ]:
import json
json_path = "/content/drive/MyDrive/sosei/data1.json"
json_open = open(json_path, 'r')
l = json.load(json_open)

a = sorted(l, key=lambda x: x[2])
output = []
for i in range(30):
  output.append(a[989-i])

with open(f"/content/drive/MyDrive/sosei/data1_top30.json", 'w') as f:
  json.dump(output, f, indent=2, ensure_ascii=False)

**2つのシラバスの全パターン**

In [ ]:
import glob
from google import colab
import itertools

files = glob.glob("/content/drive/MyDrive/sosei/meishi/*")
print(len(files))
kumiawase = list(itertools.combinations(files,2))
print(len(kumiawase))
pprint.pprint(kumiawase)
for file1 in files:
    print(file1)


# **Word2Vec**

東北大学の学習済みモデル

In [ ]:
#参考 https://gist.github.com/wai-doi/f0c85d64253b35746e3ca5fb327e10b8
import gensim

path = "/content/drive/MyDrive/sosei/Pre-Trained Word2Vec Models/entity_vector/entity_vector.model.bin"
model = gensim.models.KeyedVectors.load_word2vec_format(path, binary=False)
model.similarity('C言語', '男性')


0.08955177

白ヤギコーポレーションの学習済みモデル



In [ ]:
from gensim.models.word2vec import Word2Vec

path = "/content/drive/MyDrive/sosei/Pre-Trained Word2Vec Models/latest-ja-word2vec-gensim-model/word2vec.gensim.model"
w2v_model = Word2Vec.load(path)
try:
  print(w2v_model.similarity('C言語', 'C言語'))
  print(w2v_model.similarity('c言語', 'Python'))
  print(w2v_model.similarity('八戸', '八戸'))
except KeyError:
  print(-1)

print(w2v_model.similarity('八戸', '八戸'))

FileNotFoundError: ignored

可視化

In [ ]:
# plain word2vec t-SNE Visualization
from gensim.models.word2vec import Word2Vec
import numpy as np
from sklearn.manifold import TSNE

path = "/content/drive/MyDrive/sosei/latest-ja-word2vec-gensim-model/word2vec.gensim.model"
model = Word2Vec.load(path)
word2vec_model=model

skip=0
limit=241 

vocab = word2vec_model.wv.vocab
emb_tuple = tuple([word2vec_model[v] for v in vocab])
X = np.vstack(emb_tuple)

tsne_model = TSNE(n_components=2, random_state=0,verbose=2)
np.set_printoptions(suppress=True)
tsne_model.fit_transform(X)

plain_tsne = pd.DataFrame(tsne_model.embedding_[skip:limit, 0],columns = ["x"])
plain_tsne["y"] = pd.DataFrame(tsne_model.embedding_[skip:limit, 1])
plain_tsne["word"] = list(vocab)[skip:limit]
# plain_tsne["cluster"] = idx[skip:limit] # クラスタを計算し終わったあとならここでクラスタを付与できます
# plain_tsne.plot.scatter(x="x",y="y",c="cluster",cmap="viridis",figsize=(8, 6),s=30)
plain_tsne.plot.scatter(x="x",y="y",figsize=(8, 6),s=30)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


[t-SNE] Computing 91 nearest neighbors...


# **LDA**

In [ ]:
# MeCabオブジェクトの生成
mt = MeCab.Tagger('')
mt.parse('')

# トピック数の設定
NUM_TOPICS = 3

# トレーニングデータの読み込み
# train_texts は二次元のリスト
# テキストデータを一件ずつ分かち書き（名詞、動詞、形容詞に限定）して train_texts に格納するだけ
train_texts = []
f = [
      "日本防衛 長射程重視に変質も",
      "合区で活用 参院特定枠って",
      "21議員の政治資金 使途不透明",
      "公害で死んだ娘 語り継ぐ教訓",
      "「隠れた被害者」加害者の子",
      "ふるさと納税 寄付が広告費に",
      "全国に外国人相談窓口 通訳も",
      "ぐずりにスマホ 非難に悩む親"
]
for line in f:
  text = []
  node = mt.parseToNode(line.strip())
  while node:
      fields = node.feature.split(",")
      if fields[0] == '名詞' or fields[0] == '動詞' or fields[0] == '形容詞':
          text.append(node.surface)
      node = node.next
  train_texts.append(text)

# モデル作成
dictionary = Dictionary(train_texts)
corpus = [dictionary.doc2bow(text) for text in train_texts]
lda = LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

# テストデータ読み込み
# test_texts は train_texts と同じフォーマット
test_texts = []
raw_test_texts = []
f = [
    "爆発 現場で缶100本ガス抜き",
    "爆発の瞬間 デマ動画が拡散",
    "バニラエア 来年10月運航終了",
    "コンビニごみ 店舗負担の矛盾",
    "謎めいたカエル 南米で再発見",
    "イニエスタ うつ報道の難しさ",
    "ムネリン笑顔「少し元気に」",
    "G菅野 ゴジラ超え6.5億円に"
]
for line in f:
    text = []
    raw_test_texts.append(line.strip())
    node = mt.parseToNode(line.strip())
    while node:
        fields = node.feature.split(",")
        if fields[0] == '名詞' or fields[0] == '動詞' or fields[0] == '形容詞':
            text.append(node.surface)
        node = node.next
    test_texts.append(text)


# テストデータをモデルに掛ける
score_by_topic = defaultdict(int)
test_corpus = [dictionary.doc2bow(text) for text in test_texts]

# クラスタリング結果を出力
for unseen_doc, raw_train_text in zip(test_corpus, raw_test_texts):
    print(raw_train_text, end='\t')
    for topic, score in lda[unseen_doc]:
        score_by_topic[int(topic)] = float(score)
    for i in range(NUM_TOPICS):
        print('{:.2f}'.format(score_by_topic[i]), end='\t')
    print()

# **Doc2Vec**

In [ ]:
#参考 https://yag-ays.github.io/project/pretrained_doc2vec_wikipedia/

from gensim.models.doc2vec import Doc2Vec

model_path = "/content/drive/MyDrive/sosei/Pre-Trained Doc2Vec Models/jawiki.doc2vec.dbow300d/jawiki.doc2vec.dbow300d.model"
doc1_path = "/content/drive/MyDrive/sosei/bunsho/mono_zukuri_kiso.txt"
doc2_path = "/content/drive/MyDrive/sosei/bunsho/puroguraming_1.txt"
model = Doc2Vec.load(model_path)
model.docvecs.similarity(doc1_path, doc2_path)

KeyError: ignored

# **Google Drive マウント**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
